In [2]:
# ================================
# 국민 동의 청원 AI 모델 - 예측 코드
# ================================

# 📦 1️⃣~6️⃣: 라이브러리, 모델 로딩, 데이터 로드 및 병합

import os
import pickle
import pandas as pd

print("📂 현재 작업 디렉토리:", os.getcwd())

# 2️⃣ 모델 불러오기
try:
    with open("청원_예측모델.pkl", "rb") as model_file:
        model = pickle.load(model_file)
    print("✅ 모델 불러오기 성공!")
except Exception as e:
    print(f"❌ 모델 불러오기 실패: {e}")
    exit()

# 3️⃣ 진행 중 청원 CSV 로드
file_path = "진행중국민동의청원.csv"
try:
    진행_청원 = pd.read_csv(file_path, encoding="cp949")
    print("✅ 진행 중 청원 데이터 로드 성공!")
except Exception as e:
    print(f"❌ 진행 중 청원 데이터 로드 실패: {e}")
    진행_청원 = pd.DataFrame(columns=["청원제목", "청원내용"])
    print("빈 DataFrame을 생성했습니다.")

# 4️⃣ 제출 주체 컬럼 생성
진행_청원["제출주체"] = 진행_청원["청원제목"].apply(lambda x: "국회의원" if "법안" in str(x) else "국민")
진행_청원["제출주체"] = 진행_청원["제출주체"].map({"국민": 0, "국회의원": 1})

# 6️⃣ 크롤링된 '청원 요지' 추가 병합
try:
    크롤링된_청원 = pd.read_csv("청원_처리_현황_크롤링완료.csv", encoding="utf-8")
    진행_청원 = 진행_청원.merge(크롤링된_청원[["청원제목", "청원내용"]], on="청원제목", how="left")
    print("✅ 크롤링된 청원 요지 데이터 추가 완료!")
except Exception as e:
    print(f"❌ 크롤링된 청원 요지 데이터 로드 실패: {e}")


📂 현재 작업 디렉토리: /Users/sky/VSCode/petition/backend/AI
✅ 모델 불러오기 성공!
❌ 진행 중 청원 데이터 로드 실패: [Errno 2] No such file or directory: '진행중국민동의청원.csv'
빈 DataFrame을 생성했습니다.
❌ 크롤링된 청원 요지 데이터 로드 실패: [Errno 2] No such file or directory: '청원_처리_현황_크롤링완료.csv'


✅ 크롤링된 청원 요지 데이터 추가 완료!


KeyError: '청원내용'

In [1]:
# 🧠 7️⃣~8️⃣: BERT 임베딩 생성 + 예측 + 결과 저장

from tqdm import tqdm
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

# KoBERT 로딩
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_bert = AutoModel.from_pretrained("monologg/kobert")

def get_bert_embedding(texts):
    embeddings = []
    for text in tqdm(texts, desc="BERT 임베딩 생성"):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        with torch.no_grad():
            outputs = model_bert(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)
    return np.array(embeddings)

# 입력 텍스트 결합
test_texts = (진행_청원["청원제목"] + " " + 진행_청원["청원내용"].fillna("")).astype(str).tolist()
test_embeddings = get_bert_embedding(test_texts)

# 예측
try:
    X_test = np.hstack((test_embeddings, 진행_청원[["제출주체"]].values))
    진행_청원["승인 확률"] = model.predict_proba(X_test)[:, 1] * 100
    print("✅ 승인 확률 예측 성공!")

    # 저장
    진행_청원[["청원제목", "승인 확률"]].to_csv("예측결과.csv", index=False, encoding="utf-8-sig")
    print("✅ 예측 결과 저장 완료! (예측결과.csv)")
except Exception as e:
    print(f"❌ 승인 확률 예측 실패: {e}")


The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


NameError: name '진행_청원' is not defined